# Workbook 2: What are the Distributions of Jobs by County and by Metropolitan/Micropolitan Area?

So far in Workbook 1, we've just looked at the data by census block, with each row representing one census block. We also have a geography crosswalk table, containing information about the census blocks, including information about county and zip codes. What if we wanted to look at the jobs dataset within the context of the county they were in? We'd need to somehow combine the information from these two datasets together. We do that using `JOIN`s. 

## Motivating Question

In the previous workbook, we explored a little bit of the Workplace Area Characteristic dataset as well as the Geography Crosswalk dataset. We were able to look at how jobs were in each census block, as well as how many census blocks there were in each county or metropolitan/micropolitan area. In this notebook, we're going to explore deeper into the data available to us to try to answer the question:

**What are the characteristics of the distribution of jobs by county and by metropolitan/micropolitan area? **

To answer this, we'll have to use data from multiple tables.

## <span style="color:green">Joins (VIDEO)</span>

One of the nice things about relational databases is organization using multiple tables that are linked together in some way. For example, suppose we have one table with 6 rows called **Table A**:

| blockid | C000|
|---|---|
|1|5|
|2|10|
|3|2|
|4|6|
|5|22|
|6|9|

And another table with 5 rows called **Table B**:

| blockid | CA01|
|---|---|
|2|2|
|5|4|
|6|1|
|7|2|
|8|0|

Let's say we want to combine Table A and Table B so that we have one table that contains information about `blockid`, `C000`, and `CA01`. We want to do this by matching the two tables by what they have in common, `blockid`. That is, we want a table that looks like this (let's call this **Table C**):

| blockid | C000 | CA01 |
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|

Table C has each `blockid` that was in both Table A and Table B. It also contains the appropriate values for `C000` and `CA01` corresponding to each `blockid`. This kind of matching can be quite tricky to figure out manually, since there are different numbers of rows in each table, not all of the `blockid` values match for the two tables, and there are some `blockid` values that aren't in both. Fortunately for us, SQL is well-equipped to handle this task using the `JOIN` statement.

### SQL Code and how it works

Before we begin joining, let's first connect to the database.

In [ ]:
%defaultDatasource jdbc:sqlite:lodes.db

Now that we're connected and have established a plan for how we're joining two tables together, let's take a look at the SQL code that performs this join and break it down.

    SELECT * FROM ca_wac_2015
    JOIN ca_xwalk
    ON ca_wac_2015.w_geocode = ca_xwalk.tabblk2010 
    LIMIT 1000;

Let's look at the first two lines.

    SELECT * FROM lodes.ca_wac ca_wac_2015
    JOIN ca_xwalk

Here, we want to `SELECT` each column from a data table that we get from joining the tables `ca_wac_2015` and `ca_xwalk`. The second line takes the `ca_wac_2015` table and joins the `ca_xwalk` table to it. 

We can't just mash two tables together though -- we need some way of making sure that the appropriate rows match. We do this with the third line:

    ON ca_wac_2015.w_geocode = ca_xwalk.tabblk2010

This part specifies what we're joining on. That is, what is the ID variable that is in both tables that we want to match. Notice that they don't need to be named the same in both tables, though you do need to specify what they are in each table, even if they are the same, as well as which table they are from.

If you run the full code below, you should see the first 1000 rows (because of the `LIMIT 1000`) of the joined table. You should be able to scroll through all of the variables and see that we've managed to merge the `ca_wac_2015` and `ca_xwalk` tables together according to their census block IDs.

In [ ]:
SELECT * FROM ca_wac_2015
JOIN ca_xwalk
ON ca_wac_2015.w_geocode = ca_xwalk.tabblk2010 
LIMIT 1000;

## <span style="color:red">Checkpoint: Get Census Data and Join with your Tables

## <span style="color:green">Different Types of Joins (VIDEO)</span>

We've so far done only one type of join, an inner join. This is the default join (which is why we didn't need to specify anything more in the code). However, there are different types of joins.

### Left and Right Joins in SQL


Suppose we want to look at every single census block in one table, only filling in information from the second table if it exists. We'll illustrate this using Table A and Table B from before. Recall that our `JOIN` created Table C:

| blockid | C000 | CA01 |
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|

Instead, we want to create the following table:

| blockid | C000 | CA01 |
|---|---|---|
|1|5|*null*|
|2|10|2|
|3|2|*null*|
|4|6|*null*|
|5|22|4|
|6|9|1|

Here, we've kept every single row in Table A, and simply filled in the information from Table B if it existed for that `blockid`. This is called a **LEFT JOIN**, since we're taking the table on the left (that is, Table A) and adding the information from Table B onto that. We could have also done a **RIGHT JOIN**, which does the same thing, except flipping the tables, giving us something that looks like:

| blockid | C000 | CA01|
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|
|7|*null*|2|
|8|*null*|0|

### Applying Left Joins
When might you use left or right joins? Suppose you want to know which census blocks don't have any jobs. Then, we'd want to make sure that we keep all of the census blocks in the geography crosswalk, even if they aren't present in the workplace area characteristics tables, and add in the workplace area characteristics. This would mean that any blocks with a `null` value in the `C000` column would be blocks without any jobs. 

To do this `JOIN`, we can use the `LEFT JOIN` statement.

In [ ]:
SELECT * FROM ca_wac_2015
LEFT JOIN ca_xwalk 
ON ca_wac_2015.w_geocode = ca_xwalk.tabblk2010
LIMIT 1000;

This is very similar to what we've done already with `JOIN`, except we add the word `LEFT` to it. When doing `LEFT` and `RIGHT JOIN`s, make sure to keep track of which one is first. Here, `lodes.ca_wac` comes first, so that's the table on the "left" side.

### Outer Join

An outer join keeps all unique ids, then puts `NULL` if it isn't part of that table. This is similar to a `LEFT` or `RIGHT JOIN`, except instead of only keeping all IDs from one table, it keeps them from both tables. Consider our example with Table A and Table B. We want to join them such that we get a table that looks like:

| blockid | C000 | CA01 |
|---|---|---|
|1|5|*null*|
|2|10|2|
|3|2|*null*|
|4|6|*null*|
|5|22|4|
|6|9|1|
|7|*null*|2|
|8|*null*|0|

In a way, it's like combining the `LEFT` and `RIGHT JOIN`s so that we have all information from both tables.

### Applying Outer Joins

Suppose we want to know which census blocks that contain either the residences of people with jobs in the state or the census blocks of the location of the workplace, but not both.

In [ ]:
SELECT * FROM ca_wac_2015
OUTER JOIN ca_rac_2015 
ON ca_wac_2015.w_geocode = ca_rac_2015.h_geocode 
LIMIT 1000;

This will let us see which census blocks contain values for both workplace characteristics and residence characteristics. 

## <span style="color:red">Checkpoint: Types of Joins</span>

Consider the following situations. How would you answer the question posed? What type of join should you use for each one? Which tables do you need to join? Try doing the join.

- How many census blocks in the state contain a workplace and how many don't? Recall that the geography crosswalk table contains information about the census blocks in the state.
- Which county contains the most census blocks containing a workplace?
- Which metropolitan/micropolitan area had the most census blocks containing a residence of a worker in the state?

## <span style="color:green">Using Joins With Aggregation Functions (VIDEO)</span>

Suppose we wanted to know the distribution of the total number of jobs in California by county. The employment information is in one table, and county information is in another table. We need to join them, then aggregate the jobs, by county. Even though this seems like a complicated multi-step process, we can actually do it all in one query.  Let's break it down into two parts: the join and the aggregation.

### The Join

We need to join the `ca_wac_2015` table and the `ca_xwalk` table. Since we aren't worried about counties that have no jobs, we can do an inner join. Consider the following `JOIN` (if you want to run it, I would suggest adding a `LIMIT` at the end):

In [ ]:
SELECT a.w_geocode, a.c000, b.ctyname
FROM ca_wac_2015 a
JOIN ca_xwalk b
ON a.w_geocode = b.tabblk2010;

This should provide the `JOIN` that we want, as well as the relevant columns. We need to take the table we get from the join and apply the aggregation to it. 

### The Aggregation

From our joined table, we need `GROUP BY` county, then find the `SUM()`. For now, let's call our joined table "`joinedtable`" and write the query based on this table. Since we're also interested in what counties have the most jobs, we'll order by the sum in descending order. 

    SELECT ctyname, SUM(c000)
    FROM joinedtable
    GROUP BY ctyname
    ORDER BY SUM(c000) DESC
    LIMIT 1000;

But we've actually already figured out how to get table "`joinedtable`" using the `JOIN`s above. All we need to do is put the `JOIN` in.

In [ ]:
SELECT b.ctyname, SUM(a.c000)
FROM ca_wac_2015 a
JOIN ca_xwalk b
ON a.w_geocode = b.tabblk2010
GROUP BY ctyname
ORDER BY SUM(c000) DESC
LIMIT 1000;

This gives us one long query that performs the `JOIN`, then aggregates in the way we want. 

## <span style="color:green">Creating New Tables for Future Use (VIDEO)</span>

So far, we've mostly just been exploring the data without making any changes to the database. However, there might be times when we might want to create new tables. We can do this using `CREATE TABLE`. Let's use a previous example to create a new table.

In [ ]:
CREATE TABLE joinedtable AS 
SELECT * FROM ca_wac_2015
LEFT JOIN ca_xwalk 
ON ca_wac_2015.w_geocode = ca_xwalk.tabblk2010
LIMIT 1000;

This should look mostly familiar, since everything after the first line is stuff we've already done. The first line creates a new table called `joinedtable` from the output.

This is a bit of a mess, though. We usually don't need everything from the tables that we do join, so we can choose what we keep. Let's create a new table that has just the information we need.

In [ ]:
CREATE TABLE joinedtable2 AS 
SELECT a.w_geocode AS blockid, a."C000" AS total_jobs, b.cty AS county 
FROM ca_wac_2015 a
LEFT JOIN ca_xwalk b
ON a.w_geocode = b.tabblk2010
LIMIT 1000;

First, notice that we use aliasing to help make refering to tables easier. That is, in the third and fourth lines, we put "`a`" and "`b`" after each table to give it that alias. We can then use "`a`" and "`b`" whenever we refer to either table, which makes the `SELECT` statement easier. 

Along those lines, notice that we specify which table each variable was from. If the column name is unique between the two tables (i.e. both tables don't have a column with the same name), then you don't need to specify the table as we've done. However, if they aren't unique and both tables have a variable with that name, you need to specify which one you want.

Lastly, we've made the table easier to read by changing the name of the variable in the new table, using `AS` in the `SELECT` part of the query. 

### Dropping Tables

Conversely, you can also drop, or delete, tables. We created a table in the previous section that we won't need, so let's drop it.

In [ ]:
DROP TABLE joinedtable;

You might be tempted to avoid dropping tables since it seems relatively harmless to simply not use the table anymore without dropping them. However, it is important to keep databases clean and consider the amount of space each table takes up. 

## <span style="color:red">Checkpoint: Putting It All Together</span>

Try creating a new table using a previous query. Explore this new table using some of the techniques we've learned in the class (using aggregation functions, counting the number of rows, etc.). After you're done, drop the table and check to make sure it's gone.